In [1]:
# Cell 1: Install Dependencies and Imports
# %pip install -r requirements.txt

import os
import shutil
import cv2
import pandas as pd
import numpy as np
import json
from datetime import datetime
from ultralytics import YOLO
from IPython.display import display, Image as IPImage # Use for displaying images in notebook
from utils import MODEL_DIR, MODEL_PATH, DATA_YAML_PATH, VEHICLE_CLASSES, calculate_speed_kmh, check_anomaly

print("Dependencies installed and modules imported.")

Dependencies installed and modules imported.


Cell 2: Local File Structure Setup

In [2]:
# Cell 2: Create necessary folders
os.makedirs(MODEL_DIR, exist_ok=True)
os.makedirs('data', exist_ok=True)
print(f"Directories created: {MODEL_DIR}, data")

# Assuming 'Vechile Detection.v1-version-1.yolov8/data/data.yaml' exists from unzipping
if not os.path.exists(DATA_YAML_PATH):
    print(f"CRITICAL: Dataset YAML file not found at {DATA_YAML_PATH}.")
    print("Please ensure you have the unzipped 'Vechile Detection.v1-version-1.yolov8' folder with 'data.yaml'.")
else:
    print(f"Dataset YAML found at: {DATA_YAML_PATH}")

Directories created: models, data
Dataset YAML found at: Vechile Detection.v1-version-1.yolov8/data.yaml


Cell 3: Model Check and Training Logic

In [ ]:
# Cell 3: Model Check and Training/Loading
model = None

if os.path.exists(MODEL_PATH):
    print(f"✅ Loading existing model from local disk: {MODEL_PATH}...")
    try:
        model = YOLO(MODEL_PATH)
        print("Model loaded successfully.")
    except Exception as e:
        print(f"Error loading model: {e}. Retraining a new model...")
        model = None
        
if model is None:
    print("❌ No valid model found. Training new model...")
    try:
        model = YOLO("yolov8n.pt")
        print("Training YOLOv8n model...")
        
        results = model.train(data=DATA_YAML_PATH, epochs=40, imgsz=640) 
        
        # Save the best model
        trained_model_path = os.path.join(model.trainer.save_dir, "weights", "best.pt")
        if os.path.exists(trained_model_path):
            shutil.copy(trained_model_path, MODEL_PATH)
            print(f"Model saved to local disk: {MODEL_PATH}")
            model = YOLO(MODEL_PATH) # Reload the newly trained model
        else:
            print("Failed to find trained model weights.")
            
    except Exception as e:
        print(f"An error occurred during training: {e}")
        print("Please ensure your dataset is correctly structured and 'data.yaml' is correct.")

if model:
    MODEL_ACCURACY_SCORE = "0.834"
    print(f"Model Ready! Display Accuracy Score (Placeholder): {MODEL_ACCURACY_SCORE}")

✅ Loading existing model from local disk: models\best.pt...
Model loaded successfully.
Model Ready! Display Accuracy Score (Placeholder): 0.92


Cell 4: Example Prediction (Notebook Test)

In [ ]:
# Cell 4: Test sample image
if model:
    try:
        test_image_path = "C:\\Users\\susha\\Downloads\\skoda-india-front-license-plate.jpg"
        if os.path.exists(test_image_path):
            print(f"Running inference on test image: {test_image_path}")
            
            # Predict
            results = model.predict(test_image_path, imgsz=640, conf=0.25, verbose=False)
            res = results[0]
            
            # Read image for display
            frame = cv2.imread(test_image_path)
            detections = []
            
            # Process results
            total_count = 0
            
            for box in res.boxes.data.cpu().numpy():
                x1, y1, x2, y2, conf, cls = map(int, box[:4]) + (box[4], box[5])
                cls = int(cls)
                conf = float(conf)
                
                # Draw bounding box and label
                color = (0, 255, 0)
                class_name = VEHICLE_CLASSES.get(cls, "Unknown")
                label = f"{class_name} {conf:.2f}"
                
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
                
                if cls not in [1, 2]: # Count only vehicles, not number plates
                    total_count += 1
            
            # Save the annotated image
            output_path = "test_image_annotated.jpg"
            cv2.imwrite(output_path, frame)
            
            print(f"Total Vehicles Detected: {total_count}")
            
            # Display annotated image
            display(IPImage(filename=output_path))
            print(f"Test image annotated and saved to {output_path}")

        else:
            print(f"Skipping test inference: Sample image '{test_image_path}' not found.")
            
    except Exception as e:
        print(f"Error during test inference: {e}")

Running inference on test image: C:\Users\susha\Downloads\skoda-india-front-license-plate.jpg
Error during test inference: unsupported operand type(s) for +: 'map' and 'tuple'


In [ ]:
# Cell 5: MySQL Connection Test
import mysql.connector

MYSQL_CONFIG = {
    "host": "localhost",
    "user": "root",
    "password": "*sushant143*",
    "database": "vehicle_analytics_db"
}

try:
    conn = mysql.connector.connect(**MYSQL_CONFIG)
    if conn.is_connected():
        print("✅ MySQL Connection Successful!")
        conn.close()
except mysql.connector.Error as err:
    print(f"❌ MySQL Connection Error: {err}")
    print("Please ensure MySQL server is running and the database 'vehicle_analytics_db' exists.")

✅ MySQL Connection Successful!


In [ ]:
# Cell 4: Check Model Accuracy (mAP)
if 'model' in globals() and model:
    print("Running model evaluation (validation mode) on the dataset...")
    try:
        metrics = model.val(data=DATA_YAML_PATH)
        
        # mean Average Precision (mAP) values
        map50 = metrics.box.map50 # mAP@0.5
        map_full = metrics.box.map # mAP@[0.5:0.95]
        
        print(f"\n--- Model Accuracy Metrics ---")
        print(f"mAP@0.5 (Common Accuracy Score): {map50:.4f}")
        print(f"mAP@[0.5:0.95] (Full Range Accuracy): {map_full:.4f}")
        
    except Exception as e:
        print(f"Error during model validation: {e}")
        print("Ensure the data paths in data.yaml are correct and the model is trained.")
else:
    print("Model not loaded. Please run Cell 3 first.")

Running model evaluation (validation mode) on the dataset...
Ultralytics 8.3.235  Python-3.12.4 torch-2.2.2+cpu CPU (12th Gen Intel Core i3-1215U)
val: Fast image access  (ping: 1.20.4 ms, read: 2.30.4 MB/s, size: 48.5 KB)
val: Scanning D:\Machine Learning Projects\Advanced Vechile Detection\Version - 2.0\Vechile Detection.v1-version-1.yolov8\valid\labels... 184 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 184/184 25.9it/s 7.1s0.3s
val: New cache created: D:\Machine Learning Projects\Advanced Vechile Detection\Version - 2.0\Vechile Detection.v1-version-1.yolov8\valid\labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 12/12 5.4s/it 1:054.2ss
                   all        184       1980      0.852      0.668      0.789      0.528
                     0        159       1061      0.903      0.898      0.948      0.765
                     1        104        174      0.876      0.774      0.847      0.485
       

In [ ]:
# Cell 5: Checking Accuracy (Precision, Recall, F1-Score)
if 'model' in globals() and model:
    print("Calculating per-class Precision and Recall...")
    try:
        if 'metrics' not in locals():
            metrics = model.val(data=DATA_YAML_PATH)
            
        # YOLO stores per-class metrics in the 'p' (Precision) and 'r' (Recall) attributes of the 'box' Metric object.
        per_class_precision = metrics.box.p.tolist()
        per_class_recall = metrics.box.r.tolist()
        
        f1_scores = [2 * (p * r) / (p + r) if (p + r) > 0 else 0 
                      for p, r in zip(per_class_precision, per_class_recall)]
        
        metrics_df = pd.DataFrame({
            'Class Index': list(VEHICLE_CLASSES.keys()),
            'Class Name': [VEHICLE_CLASSES.get(i, 'N/A') for i in VEHICLE_CLASSES.keys()], # Use .get for robustness
            'Precision': per_class_precision,
            'Recall': per_class_recall,
            'F1-Score': f1_scores
        })
        
        metrics_df['Precision'] = metrics_df['Precision'].apply(lambda x: f'{x:.4f}')
        metrics_df['Recall'] = metrics_df['Recall'].apply(lambda x: f'{x:.4f}')
        metrics_df['F1-Score'] = metrics_df['F1-Score'].apply(lambda x: f'{x:.4f}')
        
        print(f"\n--- Per-Class Performance Metrics ---")
        display(metrics_df)
        
    except Exception as e:
        print(f"Error retrieving detailed metrics: {e}")
        print("Ensure 'metrics' object is defined and the validation run was successful.")
else:
    print("Model not loaded. Please run Cell 3 first.")

Calculating per-class Precision and Recall...

--- Per-Class Performance Metrics ---


,Class Index,Class Name,Precision,Recall,F1-Score
0,0,Car,0.9034,0.8982,0.9008
1,1,plate,0.8763,0.7735,0.8217
2,2,blur_plate,0.8379,0.4224,0.5616
3,3,Two Wheeler,0.8702,0.8409,0.8553
4,4,Auto,0.7401,0.5213,0.6117
5,5,Bus,0.9485,0.6692,0.7847
6,6,Truck,0.7885,0.5474,0.6462
